In [31]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [33]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

In [34]:
df_torax = df_procesada.query("COD_HOSPITAL == 112103")

## 1. Análisis Sociodemográfico de los pacientes

En este apartado se quiere responder las siguientes preguntas:

- ¿Cuál es la **edad** de los pacientes que se realizan los procedimientos en el hospital del Tórax?
- ¿Cuál es el **sexo** de los pacientes que se realizan los procedimientos en el hospital del Tórax?
- ¿Cuál es la **previsión** de los pacientes que se realizan los procedimientos en el hospital del Tórax?
- ¿Cuál es la **letra de FONASA** de los pacientes que se realizan los procedimientos en el hospital del Tórax?

Para responder estas preguntas se utilizará la base de datos de GRD Público entre 2019 y 2021.

In [35]:
columnas_sociodemográficas = ["PREVISION", "SEXO", "RANGO_ETARIO", "Región y País"]

df_procesada_formato_largo = pd.melt(
    df_torax,
    id_vars=["COD_HOSPITAL", "ANIO_EGRESO", "CIP_ENCRIPTADO"] + columnas_sociodemográficas,
    value_vars=vars_procedimientos.keys(),
    value_name="procedimiento",
)

In [36]:
agrupacion_estatica = ["ANIO_EGRESO", "Región y País"]
agrupacion_variable = ["PREVISION", "SEXO", "RANGO_ETARIO"]
variable_a_contar = "procedimiento"

In [37]:
resultados_sociodemograficos = build_features.obtener_desglose_sociodemografico(
    df_procesada_formato_largo, agrupacion_estatica, agrupacion_variable, variable_a_contar
)

In [38]:
ruta_a_guardar = "../data/interim/sociodemografico/desglose_sociodemografico_proced.xlsx"

with pd.ExcelWriter(ruta_a_guardar) as file:
    for nombre_hoja, df_a_guardar in resultados_sociodemograficos.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja, index=False)

A modo de ejemplo, se quiere saber el desglose sociodemográfico de las Angioplastías (00.66) en 2019
para el Hospital del Tórax

In [42]:
resultados_sociodemograficos["global"].query("procedimiento == '00.66' and ANIO_EGRESO == 2019.0")

,ANIO_EGRESO,Región y País,procedimiento,conteo_global,llave_id
23,2019.0,"Región Metropolitana de Santiago, Chile",00.66,618,"2019.0-Región Metropolitana de Santiago, Chil..."
579,2019.0,"Región de Antofagasta, Chile",00.66,2,"2019.0-Región de Antofagasta, Chile-00.66"
690,2019.0,"Región de Arica y Parinacota, Chile",00.66,14,"2019.0-Región de Arica y Parinacota, Chile-00.66"
831,2019.0,"Región de Atacama, Chile",00.66,59,"2019.0-Región de Atacama, Chile-00.66"
1043,2019.0,Región de Aysén del General Carlos Ibáñez del...,00.66,21,2019.0-Región de Aysén del General Carlos Ibá...
1185,2019.0,"Región de Coquimbo, Chile",00.66,33,"2019.0-Región de Coquimbo, Chile-00.66"
1581,2019.0,"Región de Los Lagos, Chile",00.66,10,"2019.0-Región de Los Lagos, Chile-00.66"
1897,2019.0,Región de Magallanes y de la Antártica Chilen...,00.66,2,2019.0-Región de Magallanes y de la Antártica...
2028,2019.0,"Región de Tarapacá, Chile",00.66,9,"2019.0-Región de Tarapacá, Chile-00.66"
2228,2019.0,"Región de Valparaíso, Chile",00.66,25,"2019.0-Región de Valparaíso, Chile-00.66"


Los resultados muestran el desglose por región para las angioplastias.